# Late Payment NBA

Simple flow:
1. Load `clients` and `invoices` tables
2. Find unpaid invoices open for more than 30 days
3. Generate one NBA event per qualifying invoice
4. Optionally post events to backend

In [ ]:
from nba_calculator.utils import CalculatorUtils, EnvConfig

In [ ]:
# Configuration
ENDPOINT = EnvConfig.str("ENDPOINT", "http://127.0.0.1:8000/api/v1/internal/events/nba-calculation")
NBA_LIST_ENDPOINT = EnvConfig.str("NBA_LIST_ENDPOINT", "http://127.0.0.1:8000/api/v1/nba")
MIN_DAYS_OPEN = EnvConfig.int("MIN_DAYS_OPEN", 30)
NBA_DEFINITION_ID = EnvConfig.str("NBA_DEFINITION_ID", "def_late_payment")
SOURCE = EnvConfig.str("SOURCE", "calculator.late-payment.v1")
REQUEST_ID_PREFIX = EnvConfig.str("REQUEST_ID_PREFIX", "calc-late-payment")
DRY_RUN = EnvConfig.bool("DRY_RUN", False)  # Set false to POST events

In [ ]:
def create_event_context(client, invoice):
    context = {
        "invoice_id": invoice.get("invoice_id"),
        "client_id": client.get("client_id"),
        "invoice_amount": float(invoice.get("amount", 0.0)),
        "date_created": invoice.get("date_created"),
        "days_open": invoice.get("days_open"),
        "rule": f"unpaid invoice open > {MIN_DAYS_OPEN} days",
        "offer": "late_payment_follow_up",
    }
    return CalculatorUtils.set_context_hash(context)

def create_delete_event_context(client, invoice):
    context = {
        "client_id": client.get('id'),
        "rule": f"unpaid invoice open > {MIN_DAYS_OPEN} days",
        "offer": "late_payment_follow_up",
        "message": f"This NBA is no longer applicable to you, since the invoice with id {invoice.get("invoice_id")} is no longer open.",
        }
    
    return CalculatorUtils.set_context_hash(context)

In [ ]:
# Fetch the open invoices per client
query = f"""
INSERT QUERY HERE
"""

open_invoice_data = CalculatorUtils.execute_sql_query(query)

In [ ]:
new_events = []
for open_invoice in open_invoice_data:
    client = { key: open_invoice.get(key) for key in ["account_id", "client_id", "enterprise_number"]}
    
    # Add logic

In [ ]:
CalculatorUtils.post_events(new_events, dry_run=DRY_RUN, endpoint=ENDPOINT, request_id_prefix=REQUEST_ID_PREFIX)

In the [UI](http://localhost:8000/ui/active-nbas?client=0123456789) you should now see two events for client with enterprise_number 123456789.